In [ ]:
!pip install selenium
!pip install pandas
!pip install webdriver-manager
!pip install beautifulsoup4

In [24]:
###크롤러 세팅

import selenium
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC #selenium에서 사용할 모듈 import
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException
import gc

##############################################
maxRun = 100000 #<-한번에 돌릴양
##############################################
fileIndex = "4"  #<--여기에 파일 이름에 붙어있는 숫자 입력해 주세요 ex) 음식점0.csv 이면 0입력, 음식점_서울0 이면 _서울0
##############################################
folderPath = "./활동데이터" #<-------여기에 파일이 있는 폴더 위치 지정해주세요
##############################################
filePath = folderPath+"/서울구미"+str(fileIndex)+".csv"
data = pd.read_csv(filePath,encoding="utf-8")

In [25]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get("https://map.naver.com/v5/")
try:
   element = WebDriverWait(driver, 10).until(
       EC.presence_of_element_located((By.CLASS_NAME, "input_search"))
   ) #입력창이 뜰 때까지 대기
finally:
   pass

#찾아야하는 태그 목록들
resultListClass = "UEzoS"
#resultListTitleClass = "tzwk0"
resultListTitleClass = "P7gyV"
resultTargetTitleDiv = "YouOG"
#resultTitleClick = "#_pcmap_list_scroll_container > ul > li > div.CHC5F > a"
resultTitleClick = "#_pcmap_list_scroll_container > ul > li > div.ouxiq > a"
searchFrame = "searchIframe"
entryFrame = "entryIframe"
addressSpan = "IH7VW"
daySpan = "kGc0c"
timeDiv = "qo7A2"
telSpan = "dry01"
descSpan = "zPfVt"
reviewOpenerA = "xHaT3"
reviewSpan = "zPfVt"
ratingSvg = "GWkzU"
imgDiv = "K0PDV"
timeOpenerSelector = "#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.vKA6F > div > ul > li.SF_Mq.Sg7qM > div > a"
descOpenerSelector = "#app-root > div > div > div > div > div > div.place_section.no_margin.vKA6F > div > ul > li.SF_Mq.I5Ypx > div > a"

def search(flag):
    if flag:
        #address = row["도로명주소"].split(" ",maxsplit=1)
        address = row["주소"].split(" ",maxsplit=1)
        keyword = row["상호명"]+" "+address[1]
    else:
        driver.switch_to.default_content()
        #driver.find_element(By.CLASS_NAME,"button_clear").click()
        driver.find_element(By.CLASS_NAME,"button_clear").send_keys(Keys.ENTER)
        #address = row["도로명주소"].split(" ")
        #keyword = row["상호명"]+" "+address[1]+" "+address[2]
        keyword = row["상호명"]+" "+row["시군구주소"]
    search_box = driver.find_element(By.CLASS_NAME,"input_search")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.ENTER)
    
def checkIsList(checkCount):
    try:
        driver.switch_to.default_content()
        WebDriverWait(driver,1.5).until(
        EC.presence_of_element_located((By.ID, searchFrame))
        )
        driver.switch_to.frame(searchFrame)    
        WebDriverWait(driver,1).until(
            EC.presence_of_element_located((By.CLASS_NAME, resultListTitleClass))
        )
        driver.switch_to.default_content()
        return 0;
    except TimeoutException:
        #검색결과 바로 식당정보로 이동했을때
        try:
            driver.switch_to.default_content()
            WebDriverWait(driver,1.5).until(
            EC.presence_of_element_located((By.ID, entryFrame))
            )
            driver.switch_to.frame(entryFrame)
            WebDriverWait(driver,1).until(
                EC.presence_of_element_located((By.CLASS_NAME, resultTargetTitleDiv))
            )
            driver.switch_to.default_content()
            return 1
        #검색결과가 없을때
        except TimeoutException:
            
            #처음이면 검색어를 다시 구성해서 검색
            if(checkCount==0):
                return 2
            #검색어를 다시 구성했는데도 없으면 없는가게
            elif(checkCount==1):
                return 3
            
def getInfo(row):
    #todo
    #영업시간 가져오기
    timeStr =data.loc[idx,"영업시간"] if type(data.loc[idx,"영업시간"]) is str else ""
    telStr=data.loc[idx,"전화번호"] if type(data.loc[idx,"전화번호"]) is str else ""
    descStr=data.loc[idx,"설명"] if type(data.loc[idx,"설명"]) is str else ""
    reviewStr=data.loc[idx,"리뷰"] if type(data.loc[idx,"리뷰"]) is str else ""
    ratingStr=data.loc[idx,"별점"] if type(data.loc[idx,"별점"]) is str else ""
    imgUrl=data.loc[idx,"사진"] if type(data.loc[idx,"사진"]) is str else ""
    try:
        #driver.find_element(By.CSS_SELECTOR,timeOpenerSelector).click()
        driver.find_element(By.CSS_SELECTOR,timeOpenerSelector).send_keys(Keys.ENTER)
    except NoSuchElementException:
        pass
    try:
        #driver.find_element(By.CSS_SELECTOR,descOpenerSelector).click()
        driver.find_element(By.CSS_SELECTOR,descOpenerSelector).send_keys(Keys.ENTER)
    except NoSuchElementException:
        pass
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    try:
        WebDriverWait(driver,2).until(
            EC.presence_of_element_located((By.CLASS_NAME, addressSpan))
        )
        address = soup.find("span",addressSpan).text
    except (NoSuchElementException,TimeoutException):
        data.loc[idx,"리뷰"] = "noResult"
        print(str(idx)+". "+data.loc[idx,"상호명"]+" no result")
        return
    #if data.loc[idx,"행정동주소"].split(" ")[1] not in address:
    if data.loc[idx,"시군구주소"].split(" ")[1] not in address:
        data.loc[idx,"리뷰"] = "noResult"
        print(str(idx)+". "+data.loc[idx,"상호명"]+" no result")
        return
        
    #시간가져오기
    #time = soup.select("#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.vKA6F > div > ul > li.SF_Mq.Sg7qM > div > a > div > div > span > div")
    day = soup.find_all("span",daySpan)
    time = soup.find_all("div",timeDiv)
    if(len(day)!=len(time)):
        time = soup.find_all("span",timeDiv)
    try:
        for t in range(len(day)):
            timeStr = ""
            for br in time[t].find_all("br"):
                br.replace_with("||")
            timeStr = timeStr+ day[t].text+": "+time[t].text+"//"
    except:
        pass

    #전화번호 가져오기
    tel= soup.find_all("span",telSpan)
    try:
        for i in range(len(tel)):
            telStr =  tel[i].text+".||."
    except:
        pass
        #소개 가져오기
    desc = soup.find_all("span",descSpan)
    try:
        for i in range(len(desc)):
            for br in desc[i].find_all("br"):
                br.replace_with(".||.")
            descStr = descStr + desc[i].text+".||."
    except:
        pass
        
    #리뷰탭으로 이동
    menuIndex = len(soup.find_all("a", "_tab-menu"))
    reviewTabOpenerSelector = "#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child("+str(menuIndex-1)+")"
    #reviewTabOpenerSelector = "#app-root > div > div > div > div.place_fixed_maintab.place_stuck.place_tab_shadow > div > div > div > div > a:nth-child(
    
    try:
        #driver.find_element(By.CSS_SELECTOR,reviewTabOpenerSelector).click()
        driver.find_element(By.CSS_SELECTOR,reviewTabOpenerSelector).send_keys(Keys.ENTER)
        WebDriverWait(driver,2).until(
            EC.presence_of_element_located((By.CLASS_NAME, reviewSpan))
        )
        reviewOpeners = driver.find_elements(By.CLASS_NAME,reviewOpenerA)
        for opener in reviewOpeners:
            #opener.click()
            opener.send_keys(Keys.ENTER)
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        reviews = soup.find_all("span",reviewSpan)
    except (TimeoutException, NoSuchElementException):
        reviews = []
    finally:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
    try:
        for i in range(len(reviews)):
            reviewStr = reviewStr+ reviews[i].text +"[>*}"
    except:
        pass
        
    try:
        ratingStr = soup.find("svg",ratingSvg).next_sibling.next_sibling.text
    except:
        pass
    imgUrl = soup.find("div",imgDiv)
       
    data.loc[idx,"영업시간"] = timeStr.replace(",","").replace("\n",".||.").replace("\r","")
    data.loc[idx,"전화번호"] = telStr.replace(",","").replace("\n",".||.").replace("\r","")
    data.loc[idx,"설명"] = descStr.replace(",","").replace("\n",".||.").replace("\r","")
    data.loc[idx,"리뷰"] = reviewStr.replace(",","").replace("\n",".||.").replace("\r","")
    data.loc[idx,"별점"] = ratingStr.replace(",","").replace("\n",".||.").replace("\r","")
    data.loc[idx,"상호명"] = data.loc[idx,"상호명"].replace(",","").replace("\r","")
    data.loc[idx,"시군구주소"] = data.loc[idx,"시군구주소"].replace(",","").replace("\r","").replace("\n",".||.")
    data.loc[idx,"주소"] = data.loc[idx,"주소"].replace(",","").replace("\r","").replace("\n",".||.")
    try:
        data.loc[idx,"사진"] = imgUrl.attrs["style"].split('"')[1]
    except AttributeError:
        data.loc[idx,"사진"] = ""
    print(str(idx)+". " +data.loc[idx,"상호명"]+" find")


cnt=0
for idx,row in data.iterrows():
#for idx in range(len(data)):
    
    if cnt == maxRun:
        break
    
    if type(data.loc[idx,"리뷰"]) is str or type(data.loc[idx,"설명"]) is str or type(data.loc[idx,"전화번호"]) is str or type(data.loc[idx,"별점"]) is str or type(data.loc[idx,"사진"]) is str:
        continue
    flag = True
    checkResult = -1
    try:
        for i in range(2):     
            search(flag)
            checkResult = checkIsList(i)
            if checkResult == 0:   #결과가 리스트로 나옴
                #todo
                #맞는지 검사하고 맞으면 크롤링 진행, 틀리면 
                #선택하고
                driver.switch_to.default_content()
                driver.switch_to.frame(searchFrame)
                #driver.find_element(By.CSS_SELECTOR,resultTitleClick).click()
                driver.find_element(By.CSS_SELECTOR,resultTitleClick).send_keys(Keys.ENTER)
                driver.switch_to.default_content()
                WebDriverWait(driver,4).until(
                EC.presence_of_element_located((By.ID, entryFrame))
                )
                driver.switch_to.frame(entryFrame)
                getInfo(row)
                driver.switch_to.default_content()
                break
            elif checkResult ==1:  #결과가 바로 식당정보로 나옴
                #바로 정보뽑기
                driver.switch_to.default_content()
                WebDriverWait(driver,4).until(
                    EC.presence_of_element_located((By.ID, entryFrame))
                )
                driver.switch_to.frame(entryFrame)
                getInfo(row)
                break
            elif checkResult ==2:  #처음인데 결과가 안나옴
                flag = False
                continue
            elif checkResult ==3: #두번째인데도 결과가 안나옴
                data.loc[idx,"리뷰"] = "noResult"
                print(str(idx)+". "+data.loc[idx,"상호명"]+" no result")
    except Exception as e:
        print("error: "+str(idx)+"번 데이터",e)
        data.loc[idx,"리뷰"] = "error"
    #다음 검색을 위해 검색어 지우기
    driver.switch_to.default_content()
    driver.find_element(By.CLASS_NAME,"button_clear").send_keys(Keys.ENTER)
    #driver.find_element(By.CLASS_NAME,"button_clear").click()
    cnt+=1
    if cnt%500==0:
        print("저장")
        gc.collect()
        data.to_csv(filePath,mode='w',index=False, encoding="utf-8")
data.to_csv(filePath,mode='w',index=False, encoding="utf-8")


C:\Users\multicampus\AppData\Local\Temp\ipykernel_22576\2540774835.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


9014. 드림신사 find
9015. 토티네 find
9016. 안젤라화이트 find
9017. 뽐누왁싱 find
9018. 그루밍뷰티 find
error: 9019번 데이터 Message: element not interactable
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x0022DF13+2219795]
	Ordinal0 [0x001C2841+1779777]
	Ordinal0 [0x000D4100+803072]
	Ordinal0 [0x000FE523+976163]
	Ordinal0 [0x000FDB93+973715]
	Ordinal0 [0x0011E7FC+1107964]
	Ordinal0 [0x000F94B4+955572]
	Ordinal0 [0x0011EA14+1108500]
	Ordinal0 [0x0012F192+1175954]
	Ordinal0 [0x0011E616+1107478]
	Ordinal0 [0x000F7F89+950153]
	Ordinal0 [0x000F8F56+954198]
	GetHandleVerifier [0x00522CB2+3040210]
	GetHandleVerifier [0x00512BB4+2974420]
	GetHandleVerifier [0x002C6A0A+565546]
	GetHandleVerifier [0x002C5680+560544]
	Ordinal0 [0x001C9A5C+1808988]
	Ordinal0 [0x001CE3A8+1827752]
	Ordinal0 [0x001CE495+1827989]
	Ordinal0 [0x001D80A4+1867940]
	BaseThreadInitThunk [0x7648FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77277B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77277B2E+238]
	(No symbol) [0

9159. 하성경뷰티어겐 find
9160. 뉴스킨 no result
9161. 코코뷰티 no result
9162. 류엘메이크업 no result
9163. 라비룬뷰티 find
error: 9164번 데이터 Message: element not interactable
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x0022DF13+2219795]
	Ordinal0 [0x001C2841+1779777]
	Ordinal0 [0x000D4100+803072]
	Ordinal0 [0x000FE523+976163]
	Ordinal0 [0x000FDB93+973715]
	Ordinal0 [0x0011E7FC+1107964]
	Ordinal0 [0x000F94B4+955572]
	Ordinal0 [0x0011EA14+1108500]
	Ordinal0 [0x0012F192+1175954]
	Ordinal0 [0x0011E616+1107478]
	Ordinal0 [0x000F7F89+950153]
	Ordinal0 [0x000F8F56+954198]
	GetHandleVerifier [0x00522CB2+3040210]
	GetHandleVerifier [0x00512BB4+2974420]
	GetHandleVerifier [0x002C6A0A+565546]
	GetHandleVerifier [0x002C5680+560544]
	Ordinal0 [0x001C9A5C+1808988]
	Ordinal0 [0x001CE3A8+1827752]
	Ordinal0 [0x001CE495+1827989]
	Ordinal0 [0x001D80A4+1867940]
	BaseThreadInitThunk [0x7648FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77277B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77277B2E+23

9349. 디디에스테틱 no result
9350. 닥터하스킨 find
9351. 에스에스에스테틱뷰티 find
9352. 뷰티앤더비스트 no result
9353. 염색앤두피 find
9354. 더꼼뷰티 find
9355. 지니s뷰티 find
9356. 뷰티레드 no result
9357. 네일올랑 no result
9358. 희쁨네일 find
9359. 올가스킨 find
9360. 유기농스킨케어 find
9361. 미미속눈썹 find
9362. 시즌포유네일 find
9363. 윤이뷰티 find
9364. 네일의진수 find
9365. 미소뷰티 find
9366. show find
9367. 뷰티리니 no result
9368. 뉴터치뷰티메디코스 no result
9369. 미애네일 find
9370. 수앤미피부바디 no result
9371. 아쇼니네일 find
9372. 진영뷰티 no result
9373. 잇츠탠 find
9374. 이지은에스테틱 find
9375. 휴족시간 find
9376. 메소드 find
9377. 이향피어메디3호 no result
9378. 혜인피부관리실 no result
9379. 뷰티창조공간연구소 no result
9380. 뷰티풀마인 no result
9381. 출장테라피힐링 no result
9382. 팀버네일 find
9383. 네일숲 find
9384. 제이라인 find
9385. 뷰티원 find
9386. 코스모스뷰티 no result
9387. 경이로운뷰티 find
9388. 도이뷰티 no result
9389. 이수피부테라피 find
9390. 뷰티엘라 find
9391. 서울미인피부관리 find
9392. 키미키미EYELASH find
9393. 어라운더스킨 find
9394. 네일앤쥴리 no result
9395. 예쁜미니 find
9396. 지니왁싱 find
9397. 블라유네일 find
9398. 체형관리 find
9399. 제이테라피 no result
9400. 제시뷰티 find
9401. 이플에스테틱 fi

9605. 소피아테라피 no result
9606. 란테라피 find
9607. 스파드갤러리 no result
9608. 두피탈모전문센터 no result
9609. 공블리네일쌀롱 no result
9610. 아랑네일 find
9611. 에블테라피 find
9612. 스텔라왁싱 find
9613. 네일701 find
9614. 네일은빛 find
9615. 오네뷰티 find
9616. 올라인뷰티샵 find
9617. 우리스킨케어 find
9618. 속눈썹달다 find
9619. 솔뷰티끄 find
9620. 일공팔네일 no result
9621. 네일템 no result
9622. 뉴페이스 no result
9623. 씨엘로 find
9624. 브론즈가든 find
9625. 라라샵 find
9626. 아나덴슈가링왁싱 find
9627. 하우스왁싱 find
9628. 바비스칼프 no result
9629. 찐태닝 find
9630. 뷰티매직 find
9631. 아나덴슈가링왁싱 find
9632. 제이스타일 find
9633. 업클로즈뷰티&왁싱 find
9634. 유아즈뷰티왁싱 find
9635. 유멜리아 find
9636. 마이슈가왁싱 find
9637. 소녀살롱 no result
9638. 아이오마강남 find
9639. 블레스태닝 find
9640. 올랑올랑 find
9641. 브네샵 find
9642. 모습 find
9643. 정원네일 find
9644. 아리뷰티 no result
9645. 네일감성 find
9646. 네일해 find
9647. 뷰티스투디오 find
9648. 라무르뷰티 find
9649. 네일누리 find
9650. 블링블링 find
9651. 가회네일 no result
9652. 네일즐리 find
9653. 이네일그상점 no result
9654. 포엠뷰티 find
9655. 신사아트랩 find
9656. 뷰티네일소리 find
9657. 비엠피부샵 find
9658. 트리버에스테틱 find
9659. 더브로우뷰티 no result
9660

...

11016. 브뉴네일 find
11017. 달리아스파 find
11018. 네일_A find
11019. 박수정에스테틱 find
11020. 리엠Li.M find
11021. 릴렉스 find
11022. 네일의달 find
11023. 흠평체형관리 no result
11024. 레브로우왁싱 find
11025. 휴마사지 find
11026. 네일바이은 find
11027. 샤르아이래쉬CHA-RE find
11028. 아르떼테라피 find
11029. 블루왁싱 no result
11030. 온화한 find
11031. 여리한다이어트 find
11032. 하늘빛에스테틱2 no result
11033. 약손체형테라피 find
11034. 엘스웨디시 no result
11035. 기린네일 find
11036. 버터볼네일 find
11037. 뷰네일 find
11038. 바요네일 find
11039. 황실체형관리 no result
11040. 네일롤롤 find
11041. 희에스테틱 find
11042. 이네일 no result
11043. 러버블뷰티랩 find
11044. 피어나왁싱 find
11045. 레옹테라피&왁싱 no result
11046. 슈가테라피 no result
11047. 뷰티홀릭서울 find
11048. 토리네일 find
11049. 럽네일 find
11050. 완벽한체형관리 no result
11051. 아른네일 find
11052. 몽떼네일 find
11053. 결이고은에스테틱 no result
11054. 네일더모던 find
error: 11055번 데이터 Message: element not interactable
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x0022DF13+2219795]
	Ordinal0 [0x001C2841+1779777]
	Ordinal0 [0x000D4100+803072]
	Ordinal0 [0x000FE523+976163]
	

11166. 보니타왁싱 find
11167. 88뷰티존 no result
11168. 주주네일 find
11169. 황후힐링숲 find
11170. 네일스토리 find
11171. 박세영피부비만 find
11172. 한독비만관리 find
11173. 한불피부관리사 find
11174. 수카패 find
11175. 한독피부비만 find
11176. 새치미네일 no result
11177. 르네셀 find
11178. 가꿈미스킨엔바디 no result
11179. 팀혼코스피트 no result
11180. S라인피부관리샵 find
11181. S라인 find
11182. 에바네일 no result
11183. 라라핏 find
11184. 감동스킨앤바디 no result
11185. 황제남성쑥뜸방 no result
11186. 상이네일 no result
11187. 리즈뷰티 find
11188. 제시네일 find
11189. 안나스킨바디 find
11190. 아이뽀샵 find
11191. 감성네일 find
11192. 신데렐라네일 find
11193. 손끝애네일 find
11194. 미인뷰티샵 find
11195. 눈썹공주 find
11196. 예신다이어트샵 no result
11197. 베버힐앤선아에스테틱 no result
11198. 이니네일 find
11199. 터치메이크업이쁘지애 find
11200. 지오피부관리샵 find
11201. 더올네일 find
error: 11202번 데이터 Message: element not interactable
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x0022DF13+2219795]
	Ordinal0 [0x001C2841+1779777]
	Ordinal0 [0x000D4100+803072]
	Ordinal0 [0x000FE523+976163]
	Ordinal0 [0x000FDB93+973715]
	Ordinal0 [0x0011E7F

11355. 크라이오랩 no result
11356. 마루네일 no result
11357. 한불피부관리실 find
11358. 밝은뷰티 no result
11359. 헤나교실 find
11360. 뷰티위드 find
11361. 킹스파 no result
11362. 그라운드네일 find
11363. 네일에반하다 find
11364. 미샵네일아트 find
11365. 홍네일 find
11366. 퀸즈네일 find
11367. 아람스킨케어 find
11368. J네일 find
11369. 아나덴슈가링왁싱구미산동확장단지점 find
11370. 민에스테틱 no result
11371. 황금손네일아트 find
11372. 네일도화 find
11373. 롯데하이마트 find
11374. 자우버피부비만관리실 find
11375. 한불테라피피부관리실 no result
11376. 양스에스테틱 find
11377. 두피탈모 find
11378. 힐링스파이 find
11379. 미쁨뷰티 find
11380. 멋진인생에스테틱 find
11381. 제이헤어앤네일 find
11382. 더네일 find
11383. 아나덴 find
11384. 아는동생네일샵 no result
11385. 라라다움 no result
11386. 왁싱플래너 no result
11387. 뷰티첼리 find
11388. 더끌림아이래쉬 find
11389. 홍스뷰티 find
11390. 더블유에스테틱 find
11391. 퀸네일 find
11392. 퀸즈네일샵 find
11393. 이영미피부샵 no result
11394. 마틸다 find
11395. 제이스킨앤드바디 no result
11396. 어라운드네일 find
11397. 넌지금네일이땡긴다 find
11398. 졸리네일 find
11399. 뷰티라운지 find
11400. 이연화피부관리실 no result
11401. 리아스킨앤바디 find
11402. 네일하는언니 find
11403. 힐링 find
11404. 한불에스테틱 find
11405. 케어셀

11590. 네일정원 find
error: 11591번 데이터 Message: element not interactable
  (Session info: chrome=105.0.5195.127)
Stacktrace:
Backtrace:
	Ordinal0 [0x0022DF13+2219795]
	Ordinal0 [0x001C2841+1779777]
	Ordinal0 [0x000D4100+803072]
	Ordinal0 [0x000FE523+976163]
	Ordinal0 [0x000FDB93+973715]
	Ordinal0 [0x0011E7FC+1107964]
	Ordinal0 [0x000F94B4+955572]
	Ordinal0 [0x0011EA14+1108500]
	Ordinal0 [0x0012F192+1175954]
	Ordinal0 [0x0011E616+1107478]
	Ordinal0 [0x000F7F89+950153]
	Ordinal0 [0x000F8F56+954198]
	GetHandleVerifier [0x00522CB2+3040210]
	GetHandleVerifier [0x00512BB4+2974420]
	GetHandleVerifier [0x002C6A0A+565546]
	GetHandleVerifier [0x002C5680+560544]
	Ordinal0 [0x001C9A5C+1808988]
	Ordinal0 [0x001CE3A8+1827752]
	Ordinal0 [0x001CE495+1827989]
	Ordinal0 [0x001D80A4+1867940]
	BaseThreadInitThunk [0x7648FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77277B5E+286]
	RtlGetAppContainerNamedObjectPath [0x77277B2E+238]
	(No symbol) [0x00000000]

11592. 오브마이 find
error: 11593번 데이터 Message: element 

11723. K 스테이 게스트하우스 명동 1호점 no result
11724. 24게스트하우스 서울시청점 find
11725. 북촌 인우하우스 한옥스테이 find
11726. 서울 담소정 게스트하우스 find
11727. 서울 순라 게스트하우스 find
11728. 명동 블루보트 게스트하우스 find
11729. 서울 댓하우스 게스트하우스 find
11730. 서울 돈키호테 게스트하우스 find
11731. 서울 벙크 게스트하우스 find
11732. 서울 드림 게스트하우스 find
11733. 종로 연당 한옥스테이 no result
11734. 서울 스타 게스트하우스 find
11735. 서울 빅존스 플레이스 게스트하우스 find
11736. 서울 아띠 게스트하우스 find
11737. 종로 미담 한옥스테이 find
11738. 서울 우 게스트하우스 find
11739. 홍대 버즈네스트 호스텔 find
11740. 홍대 더원 호스텔 no result
11741. 서울 오요호스텔 동대문1 no result
11742. 서울 북한산글램핑 find
11743. 을지로 미오2 디자인레지던스 find
11744. 서울 호스텔 하루 find
11745. 강남 류 게스트하우스 find
11746. 영등포 다다름 게스트하우스 find
11747. 홍대 스타일 게스트하우스 find
11748. 57명동 호스텔 find
11749. 홍대 롬바드 게스트하우스 find
11750. 신촌 타임트래블러 릴렉스 게스트하우스 find
11751. 서울 모꼬지 게스트하우스 find
11752. 서울 남산길 하우스 find
11753. 용산 하우재 게스트하우스 find
11754. K 게스트하우스 동대문 프리미엄 find
11755. 홍대 코코아 게스트하우스 find
11756. 서울 다산 하우스 find
11757. 종로 세화 호스텔 find
11758. 홍대 화이트테일 게스트하우스 find
11759. 24게스트하우스 명동타운점 no result
11760. 종로 남현당 한옥스테이 no

In [ ]:
####크롤링 한거까지 저장, 크롤링 끝내고 꼭 눌러주세요 ####
###############################################################


data.to_csv(filePath,mode='w',index=False, encoding="utf-8")